# Introduction
* Model calibration is the practice of **fitting model parameters to observed prices in the market**
* The problem at hand here is **calibrating the general market model to fit market quotes for vanilla (European) options**

# Why Calibration?
* In a *narrow sense*, market incompleteness implies that a given derivative cannot be replicated perfectly by trading in the underlying.
* In the *wider sense* - admitting jumps - market incompleteness implies a given derivative cannot be replicated perfectly even if trading in all assets (underlying and derivatives) is allowed.
* Tankov and Voltchkova - "Since typically the jump size is not known in advance, the risk associated to jumps cannot be hedged away completely: we are in an incomplete market. In this setting, the hedging becomes an approximation problem: instead of *replicating* an option, one tires to *minimze* the residula hedging error."

# Incomplete Market Example - Motivation for Calibration
* Consider a simple market with two dates ${t \in [0, 1]}$ and three states of the economy at t=1 that occur with equal probability.
* There are two assets, a bond that pays ${B_{1} = 10}$ and ${B_{0} = 10 = (1+r)^{-1}}$ meaning r = 0. Second, a risky security which costs ${S_{0} = 10}$ and pays tomorrow ${S_{1} = }$\begin{pmatrix} 20 \\ 10 \\ 0 \end{pmatrix}
* Because the number of future states is greather than the number of available assets, the system is underdetermined and not every payoff can be replicated.
* Because of this, the system is non-deterministic but admits a range of prices consistent with the risk-neutral probability measure with upper and lower bounds of occurrence.
* This is an unsatisfactory answer for quants - as it admits multiple prices for a derivate. To resolve the problem, **introduce calibration**
* The key idea here is that **there are so many possible risk-neutral probability measures, you must ask the market for the right one**

# Which Model to Pick? - this is what to do for the exercise
* M76: Not capable for multiple strikes and maturities - but works well for short maturities and a subset of prices. **(M76 is in chapter 8)**
* H93: Capable of replicating longer maturieis as well.
* The takeaway: **When calibrating longer maturities, you need a jump component and stochastic volatility**
* B96 is the simplest model that admits both jumps and stochastic volatility.
* **What about short rates?** - the impact of the short rate is negligible for shorter maturities, but matter in longer equity derivatives.
* Usually, the CIR85 stochastic short rate model is used

# Which Objective Function to Pick?
* Because this is an **optimization problem, not a discrete problem**, we need a loss function.
* Three candidates:
    1. Mean Squared Error of the price difference ${\min_{p} \frac{1}{N} \sum_{n=1}^N (C_{n}^{*} - C_{n}^{mod}(p))^2}$ for options 1 - N given parameter vector p
    2. Mean squared error of the relative price difference (log/%) ${\min_{p} \frac{1}{N} \sum_{n=1}^N (\frac{(C_{n}^{*} - C_{n}^{mod}(p))}{C_{n}^{*}})^2}$
    3. Mean Squared error of the implied vol difference ${\min_{p} \frac{1}{N} \sum_{n=1}^N (\sigma_{n}^{*} - \sigma_{n}^{mod}(p))^2}$
* Market implied volatilities calibrates to market while model implied volatitilies calibrates to model given observed prices.
* For our purposes, we'll weight the difference between market and model volatilities by the vega of the price, which makes more vol-sensitive options (i.e. those at the money and longer maturity) have a higher factor in optimization : ${\min_{p} \frac{1}{N} \sum_{n=1}^N (\sigma_{n}^{*} - \sigma_{n}^{mod}(p)^ * \frac{\partial C_{n}}{\partial \sigma_{n}}) ^2}$

# Which Options to Use for Calibration
* At least 3: **One ATM, one ITM, one OTM**, because this is the minimum needed to calibrate ATM volatility, smile slope, convexity
* How far ITM/OTM? :  ${K^{ATM} +/- \sigma^{ATM} \sqrt{T}}$

# What Optimization Algorithm?
* A four step multi-optimization process
    1. Calibrate stochastic short rate
    2. Calculate (H93) stochastic vol via global to local optimization
    3. Calibrate jump component (BCC97) with stochastic volatility as an input
    4. Caluculate stochastic volatility of (BCC97) via local optimization, with all other calibrations as an input - letting jumps freely vary once more
* Takeaway: **Let's Calibrate Each Component Against Market Data!**

# Calibration - Short Rate Theoretical
* The governing stochastic differential equation is ${dr_{t} = \kappa(\theta_{r} - r_{t})dt + \sigma_{r} \sqrt{r_{t}} d Z_{t}}$
* Kappa is mean reversion factor, theta is the mean, and Z is a Wiener process
* Calibration aims to the set parameter vector ${\alpha = \{\kappa_{r}, \theta_{r}, \sigma_{r}, r_{0}\}}$ to minimize the difference ${\Delta f(0,t) \equiv f(0, t) - f^{CIR85}(0, t; \alpha)}$ e.g. the difference between market implied rate and model implied rate. Denoting ${B_{t}(T), t < T}$ as the the price of a zero maturing at time T and paying par at that date, then the instantaneous forward rate at time t for time T is ${f(t, T) \equiv - \frac{\partial B_{t}T}{\partial T}}$
* The yield curve version of this is ${B_{t}(T) = exp(- \int_{t}^{T} f(t,s)ds)}$
* With these in mind, the CIR85 formulation is ${f^{CIR85}(0, t; \alpha) = \frac{\kappa_{r} \theta_{r} (e^{\gamma t}) - 1}{2 \gamma + (\kappa_{r} + \gamma) (e^{\gamma t) - 1})} + r_{0} \frac{4 \gamma^2 e^{\gamma t}}{(2 \gamma + (\kappa_{r} + \gamma) (e^{\gamma t} - 1))^2}}$, given ${\gamma \equiv \sqrt{\kappa_{r}^2 + 2 \sigma_{r}^2}}$
* The affine form of the price of a zero at time t is ${B_{t}(T) = \alpha(t,T) e^{-b(t, T) E_{0}^{Q}r_{t}}}$ with ${\alpha(t, T) \equiv [\frac{2\gamma exp(0.5(\kappa_{r} + \gamma)(T - t)}{2\gamma + (\kappa_{r} + \gamma)(e^{\gamma(T - t)} - 1}]^{\frac{2\kappa_{r}\sigma_{r}}{\sigma_{r}^2}}}$ and ${b(t, T) \equiv \frac{2(e^{\gamma (T - t)} - 1)}{2\gamma + (\kappa_{r} + \gamma) (e^{\gamma (T - t)} - 1)} }$

# Calibration - Short Rate Empirical
* Forwards are rarely quoted directly in the market - but yields, reference, and swap rates are (e.g. UST yields, Bunds, LIBOR, Euribor, OIS, Eonia spot/swap)
* The correspondence between yields on a zero and forwards is ${f(0, T) = Y(0, T) + \frac{\partial Y(0, T)}{\partial(T)} * T}$ with ${Y(0, T)}$ as the yield of a bond today.
* The continuous version of this solves the equation ${Y(0, T) = \frac{log B_{T}(T) - log B_{0}(T)}{T}}$
* Finally, the value of the bond normalized to 1 gives us ${Y(0, T) = -\frac{log(B_{0}(T))}{T}}$
* Calibration steps:
    1. Plot the market-observable forward rates
    2. Interpolate a cubic rate curve
    3. Minimize the mean square error at seleted discrete points: Given a fixed ${r_{0}}$ and equidistant spacing on ${[0, T]}$ by ${\Delta t}$ with ${M \equiv T/\Delta t}$, the algo is ${\min_{\alpha} \frac{1}{M} \sum_{m}^{M} (f(0, m \Delta t; \alpha) - f^{CIR85}(0, m \Delta t; \alpha))^2}$

# Calibration - Equity Component
* Now that we have the calibrated short rate, we can apply the Fourier transform.
* The fundamental formula here is: ${C_{0}(K, T) = S_{0} - B_{0}(T) \sqrt{S_{0} K} \frac{1}{\pi}} * \int_{0}^{\infty} Re[e^{-iuk} \phi (u - i/2, T)] \frac{du}{u^2 + 1/4}$ 
* ${k \equiv log(S_{0}/K)}$ and ${\phi}$ is the characteristic function of the model at hand.
* ${B_{0}(T)}$ is the discount factor
* Two characteristic functions are used here, and we take their product: First, the jump component from M76: ${\phi^{M76K}(u, T) = exp((i u \omega + \lambda (e^{i u \mu_{J} - u^2 \delta^2/2} - 1))T}$ with the drift correction ${\omega}$ as ${\omega \equiv - \lambda (e^{\mu_{J} + \delta^2/2} - 1)}$ and second the H93 model ${\phi^{H93}(u, T) = e^{H_{1}(u, T) + H_{2}(u, T) v_{0}}}$
* The H93 term needs its own definitions:
    1. ${c_{1} \equiv \kappa_{v} \theta_{v}}$
    2. ${c_{2} \equiv - \sqrt{(\rho \sigma_{v} u i - \kappa_{v})^2 - \sigma_{v}^2 (-ui - u^2)}}$
    3. ${c_{3} \equiv \frac{\kappa_{v} - \rho \sigma_{v} ui + c_{2}}{\kappa_{v} = \rho \sigma_{v} ui - c_{2}}}$
    4. ${H_{1}(u, T) = r_{0, T} uiT + \frac{c_{1}}{\sigma_{v}^2} \{ (\kappa_{v} - \rho \sigma_{v} ui + c_{2})T - 2log [\frac{1 - c_{1} e^{c^{3} T} }{1 - c_{3}}]\}}$
    5. ${H_{2}(u, T) \equiv \frac{\kappa_{v} - \rho \sigma_{v} ui + c_{2}}{\sigma_{v}^2} [\frac{1 - e^{c_{2} T}}{1 - c_{3} e^{c_{2} T}}]}$

In [4]:
#
# Valuation of European Call and Put Options
# Under Stochastic Volatility and Jumps
# 09_gmm/BCC_option_valuation.py
#
# (c) Dr. Yves J. Hilpisch
# Derivatives Analytics with Python
#
import numpy as np
from scipy.integrate import quad
import warnings
warnings.simplefilter('ignore')

#
# Example Parameters B96 Model
#
## H93 Parameters
kappa_v = 1.5
theta_v = 0.02
sigma_v = 0.15
rho = 0.1
v0 = 0.01

## M76 Parameters
lamb = 0.25
mu = -0.2
delta = 0.1
sigma = np.sqrt(v0)

## General Parameters
S0 = 100.0
K = 100.0
T = 1.0
r = 0.05

def H93_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    ''' Valuation of European call option in H93 model via Lewis (2001)
    Fourier-based approach.

    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial level of variance

    Returns
    =======
    call_value: float
        present value of European call option

    '''
    int_value = quad(lambda u: H93_int_func(u, S0, K, T, r, kappa_v,
                                            theta_v, sigma_v, rho, v0),
                     0, np.inf, limit=250)[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) /
                     np.pi * int_value)
    return call_value


In [10]:
'''
Calibration Functions
'''
def H93_error_function(p0, i):
    '''
    Error function for parameter vector calibration from Lewis (2001)
    Fourier approach
    
    Params
    ===
    kapp_v: mean reversion factor
    theta: long-run mean of variance
    sigma: vol of variance
    rho: volatility/underlying correlation
    v0: instantaneous variance
    
    Returns
    ===
    MSE of model to market
    '''
    i, min_MSE
    kappa_v, theta_v, sigma_v, rho, v0 = p0 # Unpack initial guess tuple
    
    if kappa_v < 0.0 or theta_v < 0.005 or sigma_v < 0.0 or rho < -1.0 or rho > 1.0:
        return 500.0 # Max Error for invalid params
    if 2 * kappa_v * theta_v < sigma_v **2:
        # If mean reversion factor * long run mean of variance is less than vol^2,
        # Max error, system going supercrit
        return 500.0 
    se = []
    for row, option in options.iterrows():
        model_alue = H93_call_value(S0, option['Strike'], option['T'], option['r'],
                                   kappa_v, theta_v, sigma_v, rho, v0)
        se.append(model_value - option['Call'] ** 2) # Add MSE
    MSE = sum(se) / len(se)
    min_MSE = min(min_MSE, MSE) # Was this run btter than the previous run?
    
    if i % 25 == 0:
        print(f'{i:4d} | {np.array(p0)} | {MSE: 7.3f} | {min_MSE: 7.3f}')
    i += 1
    return MSE
    

In [16]:
p0_values = ((2.5, 10.6, 5.0), # Kappa
            (0.01, 0.041, 0.01), # Theta
            (0.05, 0.251, 0.1), # Sigma
            (-0.75, 0.01, 0.25), # Rho
            (0.01, 0.031, 0.01)) # Initial Vol
def H93_calibration_full(p0_values, i):
    '''
    Calibrate H93 Stochastic Volatility model to market quotes.
    
    Taking the standard approach to optimization, brute force to get to a sensible
    global region to explore, then perform a convex minimization on that region
    '''
    p0 = brute(H93_error_function(p0_values, i), Finish=None)
    
    opt = fmin(H93_error_function, p0,
              xtol=0.000001, ftol=0.000001,
              maxiter=750, maxfun=900)
    np.save('opt_sv', np.array(opt))
    return opt

In [18]:
def H93_calculate_model_values(p0):
    '''
    Calculates all model values given the optimal parameter vector p0.
    '''
    kappa_v, theta_v, sigma_v, rho, v0 = p0
    values = []
    for row, option in option.iterrows():
        model_value = H93_call_value(S0, option['Strike'], option['T'],
                                    option['r'], kappa_v, theta_v, sigma_v, rho, v0)
        values.append(model_value)
    return np.array(values)

In [11]:
import sys
import math
import numpy as np
np.set_printoptions(suppress=True, formatter={'all': lambda x: '%5.3f' % x})
import pandas as pd
from scipy.optimize import brute, fmin, minimize
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

# data = Pull in Yves data here
S0 = 100 # Replace
r0 = 0.05 # Constant Short Rate


def h93_calibration(data, short_rate):
    '''
    Select options to use for calibration
    '''
    tol = 0.02 # Select those options 2% above/below ATM
    options = data[(np.abs(data['Strike']) - S0 / S0) < tol] 
    i = 0 # Optimization iteration pointer
    '''
    Add time to maturity and short rate
    '''
    for row, option in oprtions.iterrows():
        T = (option['Maturity'] - option['Date']).days / 365. # Day count
        options.ix[row, 'T'] = T
        options.ix[row, 'r'] = -math.log(short_rate) / T
        
    
    

# Comparison of Implied Volatilities
* Now we can calculate an implied volatility from the models we have

In [19]:
def calculate_implied_volatilities(data, r):
    options = data
    for row, option in options.iterrows():
        T = option(['Maturity'] - option['Date']).days / 365.
        B0T = r
        r = -math.log(r) / t
        call = call_option(S0, option['Strike'], option['Date'], option['Maturity'],
                          option['r'], 0.1)
        options.ix[row, 'market_iv'] = call.imp_vol9(option['Call'], 0.15)